In [ ]:
import os
import tifffile
import matplotlib.pyplot as plt
from pathlib import Path
import random
import numpy as np

# Notebook-friendly plot display
%matplotlib inline

# Use every frame in test set (stride of 1 frame)
SAMPLE_FRAME_INTERVAL = 1

# Set paths
input_dir = Path("raw-data")
output_base = Path("../full-test-data")

# Create output directories
for subset in ["train", "val", "test"]:
    (output_base / subset / "A").mkdir(parents=True, exist_ok=True)
    (output_base / subset / "B").mkdir(parents=True, exist_ok=True)

In [ ]:
# Process files from already-split directories
BF_CMAP = "gray"    # Bright-field colormap
FL_CMAP = "magma"   # Fluorescence colormap

# Set a reference frame to normalize against frame flickering in bright-field
ref_bf_mean = None
ref_bf_std = None
for ref_tif_path in sorted((input_dir / "train").glob("*.tif")):
    try:
        with tifffile.TiffFile(ref_tif_path) as tif:
            arr = tif.asarray()
        bright_field_index = 0
        if arr.ndim == 4:
            ref_bf = arr[0, bright_field_index]
        elif arr.ndim == 3 and arr.shape[0] == 2 and arr.shape[1:] == (512, 512):
            ref_bf = arr[bright_field_index]
        else:
            continue
        ref_bf = ref_bf.astype(np.float32)
        ref_bf_mean = ref_bf.mean()
        ref_bf_std = ref_bf.std()
        print(f"Using TRAIN ref '{ref_tif_path.name}'. Mean={ref_bf_mean:.2f}, Std={ref_bf_std:.2f}")
        break
    except Exception as e:
        print(f"Skipping {ref_tif_path.name}: {e}")
        continue

if ref_bf_mean is None or ref_bf_std is None:
    raise RuntimeError("No valid bright-field reference found in training set")

# Safety check to prevent divide by zero
if ref_bf_std < 1e-6:
    raise RuntimeError("Erroneous bright-field reference standard deviation calculation")

# Visual range to match reference image
BF_VMIN = max(0, ref_bf_mean - 2*ref_bf_std)
BF_VMAX = min(65535, ref_bf_mean + 2*ref_bf_std)

for current_split in ["test"]:
    print(f"\n=== Processing split: {current_split} ===")
    file_list = sorted((input_dir / current_split).glob("*.tif"))
    print(f"Found {len(file_list)} TIFFs in {current_split}")

    for tif_path in file_list:
        print(f"Opening {tif_path.name}...")
        try:
            with tifffile.TiffFile(tif_path) as tif:
                arr = tif.asarray()

            bright_field_index = 0
            fluorescence_index = 1

            # Work out indexing helpers
            if arr.ndim == 4:
                n_frames = arr.shape[0]
                if arr.shape[1] <= 4:                    # (frames, channels, H, W)
                    get_bf = lambda i: arr[i, bright_field_index]
                    get_fl = lambda i: arr[i, fluorescence_index]
                else:                                     # (channels, frames, H, W)
                    n_frames = arr.shape[1]
                    get_bf = lambda i: arr[bright_field_index, i]
                    get_fl = lambda i: arr[fluorescence_index, i]
            elif arr.ndim == 3 and arr.shape[0] == 2 and arr.shape[1:] == (512, 512):
                n_frames = 1
                get_bf = lambda i: arr[bright_field_index]
                get_fl = lambda i: arr[fluorescence_index]
            else:
                print(f"ERROR: {tif_path} has unsupported shape {arr.shape}")
                continue

            # Process frames
            for frame_index in range(0, n_frames, SAMPLE_FRAME_INTERVAL):
                bf_image = get_bf(frame_index).astype(np.float32)

                # Apply mean and std from reference
                mu = bf_image.mean()
                sd = bf_image.std()
                bf_image = (bf_image - mu) * (ref_bf_std / (sd + 1e-8)) + ref_bf_mean

                # Clip to possible values
                bf_image = np.clip(bf_image, 0, 65535)

                fl_image = get_fl(frame_index)

                # Sanity check
                if (
                    bf_image is None or fl_image is None or
                    bf_image.shape != (512, 512) or fl_image.shape != (512, 512)
                ):
                    print(f"ERROR: Skipping frame {frame_index} from {tif_path}")
                    continue

                base_name = f"{tif_path.stem}_{frame_index:04d}"
                split_A = output_base / current_split / "A"
                split_B = output_base / current_split / "B"

                # Save without augmented copies
                plt.imsave(split_A / f"{base_name}.png",
                           bf_image, cmap=BF_CMAP, vmin=BF_VMIN, vmax=BF_VMAX)
                plt.imsave(split_B / f"{base_name}.png",
                           fl_image, cmap=FL_CMAP)

        except Exception as e:
            print(f"ERROR processing {tif_path}: {e}")